<a href="https://colab.research.google.com/github/parkrye/Python/blob/main/Others/AICon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 데이터 확인

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics 
from sklearn.impute import KNNImputer
from lightgbm import LGBMClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:90% !important;}</style>"))
from sklearn.preprocessing import LabelEncoder
import torch
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train  = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/Dacon/004/train.csv')
test  = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/Dacon/004/test.csv')

In [ ]:
#train.head()

In [ ]:
#train.info()

# 데이터 전처리

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/Dacon/004/train.csv')
data = data.drop('index', axis=1) # id 제외하고 분석
test = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/Dacon/004/test.csv')
test = test.drop('index', axis=1) # id 제외하고 분석

In [ ]:
dict = {}
count = 0
for i in data.index:
  if data.loc[i, 'country'] is None:
    data.loc[i, 'country'] = 999
  elif len(dict) == 0:
    dict[data.loc[i, 'country']] = count
    data.loc[i, 'country'] = dict[data.loc[i, 'country']]
    count += 1
  else:
    if data.loc[i, 'country'] in dict:
      data.loc[i, 'country'] = dict[data.loc[i, 'country']]
    else:
      dict[data.loc[i, 'country']] = count
      data.loc[i, 'country'] = dict[data.loc[i, 'country']]
      count += 1

In [ ]:
for i in test.index:
  if test.loc[i, 'country'] is None:
    test.loc[i, 'country'] = 999
  else:
    if test.loc[i, 'country'] in dict:
      test.loc[i, 'country'] = dict[test.loc[i, 'country']]
    else:
      dict[test.loc[i, 'country']] = count
      test.loc[i, 'country'] = dict[test.loc[i, 'country']]
      count += 1

In [ ]:
data = data.fillna(data.mean())
test = test.fillna(test.mean())

In [ ]:
#data.plot(kind='box', subplots=True, layout=(5, 14), figsize=(15, 21))
#plt.show()      

In [ ]:
def outliers_iqr(data):
    q1, q3 = np.percentile(data, [25, 75])
    # 넘파이의 값을 퍼센트로 표시해주는 함수

    iqr = q3 - q1
    lower_bound = q1 - (iqr * 1.5)
    upper_bound = q3 + (iqr * 1.5)
    
    return np.where((data > upper_bound) | (data < lower_bound))

In [ ]:
data[data['married'] == 3] = int(data['married'].mean())
data[data['familysize'] > 100] = int(data['familysize'].mean())

introelapse_index_data = outliers_iqr(data['introelapse'])[0]
testelapse_index_data = outliers_iqr(data['testelapse'])[0]
surveyelapse_index_data = outliers_iqr(data['surveyelapse'])[0]
age_index_data = outliers_iqr(data['age'])[0]
familysize_index_data = outliers_iqr(data['familysize'])[0]

data.loc[introelapse_index_data, 'introelapse'] = int(data['introelapse'].mean())
data.loc[testelapse_index_data, 'testelapse'] = int(data['testelapse'].mean())
data.loc[surveyelapse_index_data, 'surveyelapse'] = int(data['surveyelapse'].mean())
data.loc[age_index_data, 'age'] = int(data['age'].mean())
data.loc[familysize_index_data, 'familysize'] = int(data['familysize'].mean())

In [ ]:
#data.plot(kind='box', subplots=True, layout=(5, 14), figsize=(15, 21))
#plt.show()       

# 데이터 예측

## 정확도

In [ ]:
def ACCURACY(true, pred):   
    score = np.mean(true==pred)
    return score

## K-Fold

In [ ]:
def kfold(model, train, scale = False):
    cv_accuracy = []
    cv = StratifiedKFold(n_splits=5)
    
    n_iter = 0
    
    for t, v in cv.split(train, train['nerdiness']):
        
        train_cv = train.iloc[t] # 훈련용
        val_cv = train.iloc[v] # 검증용 분리

        train_X = train_cv.drop('nerdiness', axis=1)
        train_y = train_cv['nerdiness']

        val_X = val_cv.drop('nerdiness', axis=1)
        val_y = val_cv['nerdiness']
            
        model.fit(train_X, train_y)
        score = ACCURACY(val_y, model.predict(val_X))
        
        cv_accuracy.append(score)
        n_iter += 1
    return np.mean(cv_accuracy)

In [ ]:
models = [
    KNeighborsClassifier(),
    LogisticRegression(),
    DecisionTreeClassifier(),
    RandomForestClassifier(max_depth=12, min_samples_leaf=8, min_samples_split=20, n_estimators=300),
    GradientBoostingClassifier(),
    ExtraTreesClassifier(max_depth=25, n_estimators=320)
]

#print('스케일링 적용 전')
#for model in models:
#    print(f'{type(model).__name__} score: {kfold(model, data)}')

In [ ]:
train = data.drop('nerdiness', axis=1)
target = data['nerdiness']

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/Dacon/004/sample_submission.csv')

In [ ]:
model_rf = RandomForestRegressor()

model_rf.fit(train, target)
prediction_rf = model_rf.predict(test)
submission_rf = submission
submission_rf['nerdiness'] = prediction_rf.round(0).astype(int)
submission_rf.to_csv('submit_rf.csv', index = False)

In [ ]:
model_gb = GradientBoostingClassifier()

model_gb.fit(train, target)
prediction_gb = model_gb.predict(test)
submission_gb = submission
submission_gb['nerdiness'] = prediction_gb.round(0).astype(int)
submission_gb.to_csv('submit_gb.csv', index = False)

In [ ]:
model_et = ExtraTreesClassifier(max_depth=25, n_estimators=320)

model_et.fit(train, target)
prediction_et = model_et.predict(test)
submission_et = submission
submission_et['nerdiness'] = prediction_et.round(0).astype(int)
submission_et.to_csv('submit_et.csv', index = False)

In [ ]:
submission_vt = submission
for i in submission_rf.index:
  sum = 0
  sum += submission_rf.loc[i, 'nerdiness']
  sum += submission_gb.loc[i, 'nerdiness']
  sum += submission_et.loc[i, 'nerdiness']
  if sum >= 2:
    submission_vt.loc[i, 'nerdiness'] = 1
  else:
    submission_vt.loc[i, 'nerdiness'] = 0
submission_vt.to_csv('submit_final.csv', index = False)